In [1]:
import configparser
import json
import spotipy
import spotipy.util as util
import pandas as pd
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
import bamboolib

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open('../data/raw/spotify_creds.json') as f:
    spotify_creds = json.load(f)

In [3]:
client_id = spotify_creds['client_id']
client_secret = spotify_creds['client_secret']
username = spotify_creds['username']
scope = spotify_creds['scope']
redirect_uri = spotify_creds['redirect_url']

In [4]:
def connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri):
    
    client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

    if token:
        sp = spotipy.Spotify(auth=token)
    else:
        print("Can't get token for", username)
        
    return sp

In [5]:
sp = connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri)

In [6]:
def get_saved_library(sp):
    
    df_saved_tracks = pd.DataFrame()
    track_list = ''
    added_ts_list = []
    artist_list = []
    title_list = []
    popularity_list = []
    album_cover_list = []
    more_songs = True
    offset_index = 0

    while more_songs:
        songs = sp.current_user_saved_tracks(offset=offset_index)
        for song in songs['items']:
            #join track ids to a string for audio_features function
            track_list += song['track']['id'] +','
            #get the time when the song was added
            added_ts_list.append(song['added_at'])
            #get the title of the song
            title_list.append(song['track']['name'])
            #get popularity
            popularity_list.append(song['track']['popularity'])
            album_cover_list.append(song['track']['album']['images'][0]['url'])
            #get all the artists in the song
            artists = song['track']['artists']
            artists_name = ''
            for artist in artists:
                artists_name += artist['name']  + ','
            artist_list.append(artists_name[:-1])
        track_list = ''
        if songs['next'] == None:
            # no more songs in playlist
            more_songs = False
        else:
            # get the next n songs
            offset_index += songs['limit']
    #include timestamp added, title and artists of a song
    df_saved_tracks['song_title'] = title_list
    df_saved_tracks['artists'] = artist_list
    df_saved_tracks['date_added'] = added_ts_list
    df_saved_tracks['album_cover'] = album_cover_list
    df_saved_tracks['popularity'] = popularity_list
    
    return df_saved_tracks

In [7]:
df = get_saved_library(sp)

In [8]:
df

,song_title,artists,date_added,album_cover,popularity
0,Nightmare,K-391,2022-02-26T19:55:18Z,https://i.scdn.co/image/ab67616d0000b2739f4a62...,45
1,If I Ain't Got You - Acoustic,John Adams,2022-02-21T05:09:51Z,https://i.scdn.co/image/ab67616d0000b273d2f483...,54
2,If I Ain't Got You,Alicia Keys,2022-02-21T05:09:25Z,https://i.scdn.co/image/ab67616d0000b27356ff19...,83
3,Lost At Sea (Illa Illa 2),"B.I,Bipolar Sunshine,Afgan",2022-02-20T22:04:25Z,https://i.scdn.co/image/ab67616d0000b2730c6c9a...,61
4,When I'm Gone (with Katy Perry) [VIP Mix],"Alesso,Katy Perry",2022-02-20T19:24:17Z,https://i.scdn.co/image/ab67616d0000b273b5b2dd...,67
...,...,...,...,...,...
1098,Lullaby,"R3HAB,Mike Williams",2018-04-06T15:56:41Z,https://i.scdn.co/image/ab67616d0000b27342ccfe...,64
1099,Hold Me - AmPm Remix,"R3HAB,AmPm",2018-04-06T15:56:26Z,https://i.scdn.co/image/ab67616d0000b2732b245d...,20
1100,Low - R3HAB Remix,"Greyson Chance,R3HAB",2018-04-06T15:56:21Z,https://i.scdn.co/image/ab67616d0000b273f0073a...,39
1101,Ain't That Why - Skytech Remix,"R3HAB,Krewella,Skytech",2018-04-06T15:56:19Z,https://i.scdn.co/image/ab67616d0000b2734c49a1...,49
